In [119]:
# write a scraper that reads the main body of a forum post and returns the text of the post
import re
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import numpy as np


def find_answer_nodes(soup):
    """
    Extract the answer html between the question and the next question

    Args:
        soup: The soup of the question

    Returns:
        The formatted html of the answer
    
    """
    position = soup
    answer_nodes = []
    while True:
        try:
            position = position.find_next_sibling()
            if 'start' in position.attrs:
                break

            answer_nodes.append(position.decode_contents())
        except AttributeError:
            break

    return ''.join(answer_nodes).strip()


def extract_qa(item):
    """
    Extract the question and answer by an answer soup

    Args: 
        item: The soup of the qa post 

    Returns:
        A dictionary of the question and answer pairs 
    """
    title_tmp = item.select_one('li').decode_contents()
    title = title_tmp.replace('<code>', '\'').replace('</code>', '\'')

    # if 'What is the difference between tensor rank and shape?' in title:
    #     print(111)

    # Get the answer content
    answer_inner_html = find_answer_nodes(item)  # Get original html

    # Replace the code blocks
    answer_inner_html = re.sub(r'<pre><code( class="[\w-]+")?>([\s\S]*?)</code></pre>', r'```\2```', answer_inner_html)
    answer_inner_html = answer_inner_html.replace('<code>', '\'').replace('</code>', '\'')

    # Get content
    answer_soup = BeautifulSoup(answer_inner_html, "html.parser")
    answer = answer_soup.text

    return {'title': title, 'answer': answer.strip()}


def get_qa_post(post_url):
    """
    Reads the main body of a forum post and returns the text of the post.
    
    Args:
        post_url: The URL of the forum post.
    
    Returns:
        The text of the forum post.
    """

    # Read the HTML of the forum post
    response = requests.get(post_url)
    soup = BeautifulSoup(response.text, "html.parser")

    items = soup.find_all('ol')

    # Get the valid question titles
    QAs = []
    for i, item in enumerate(items):
        # Only get question pairs
        if i == 0 or 'start' in item.attrs:
            QAs.append(extract_qa(item))

    return QAs



In [170]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [120]:
# # get the forum post for 4 
# url = "https://forums.fast.ai/t/fastbook-chapter-4-questionnaire-solutions-wiki/67253"
# qa_dict = get_qa_post(url)
# # print(qa_dict)

# pd.DataFrame(qa_dict).to_csv('fastai-1.csv', index=False)

In [121]:
# load the csv file "forum pages" into a df and then loop through the pages and scrape the questions and answers
forum_urls_df = pd.read_csv('fastai-solutions-forum-pages.csv') 
forum_urls_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'fastai-solutions-forum-pages.csv'

In [ ]:

# loop through the pages and scrape the questions and answers
qa_dict = []
for i, row in forum_urls_df.iterrows():
    # print(i, row['URL'])

    # if it contains nan or is empty or is not a url, skip
    if not pd.isna(row['URL']) and row['URL'].startswith('https'):
        # print(row['URL'])
        qa_dict.extend(get_qa_post(row['URL']))

# save the questions and answers to a csv file
qadict = pd.DataFrame(qa_dict)
qadict.to_csv('fastai-2.csv', index=False)


0 https://forums.fast.ai/t/fastbook-chapter-1-questionnaire-solutions-wiki/65647
https://forums.fast.ai/t/fastbook-chapter-1-questionnaire-solutions-wiki/65647
1 https://forums.fast.ai/t/fastbook-chapter-2-questionnaire-solutions-wiki/66392
https://forums.fast.ai/t/fastbook-chapter-2-questionnaire-solutions-wiki/66392
2 https://forums.fast.ai/t/fastbook-chapter-3-questionnaire-solutions-wiki/68042
https://forums.fast.ai/t/fastbook-chapter-3-questionnaire-solutions-wiki/68042
3 https://forums.fast.ai/t/fastbook-chapter-4-questionnaire-solutions-wiki/67253
https://forums.fast.ai/t/fastbook-chapter-4-questionnaire-solutions-wiki/67253
4 https://forums.fast.ai/t/fastbook-chapter-5-questionnaire-solutions-wiki/69301
https://forums.fast.ai/t/fastbook-chapter-5-questionnaire-solutions-wiki/69301
5 https://forums.fast.ai/t/fastbook-chapter-6-questionnaire-solutions-wiki/69922
https://forums.fast.ai/t/fastbook-chapter-6-questionnaire-solutions-wiki/69922
6 nan
7 https://forums.fast.ai/t/fastboo

/usr/local/Caskroom/miniconda/base/envs/ml/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


11 https://forums.fast.ai/t/fastbook-chapter-12-questionnaire-wiki/70516
https://forums.fast.ai/t/fastbook-chapter-12-questionnaire-wiki/70516
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan


In [ ]:
# print(qadict.head())
print(len(qadict))

297


In [ ]:
# open the csv file, remove empty lines, and ensure every line has a question and answer, and ensure answers end with a '.' or ' 'ArithmeticError
qa_df = pd.read_csv('fastai-qa-cropped.csv')
qa_df = qa_df.dropna()
# if an answer does not end with a '.' or ' ' or '`', add a '.'
qa_df['answer'] = qa_df['answer'].apply(lambda x: x if x.endswith('.') or x.endswith(' ') or x.endswith('`') else x + '.')
# save the questions and answers to a csv file
qa_df.to_csv('fastai-qa-cleaned.csv', index=False)
qa_df.head()

,question,answer
0,"Based on the book of the same name, what are t...",- A set of processing units\n- A state of acti...
1,What were the two theoretical misunderstanding...,"In 1969, Marvin Minsky and Seymour Papert demo..."
2,What is a GPU?,GPU stands for Graphics Processing Unit (also ...
6,Why is it hard to use a traditional computer p...,"For us humans, it is easy to identify images i..."
7,What term do we normally use in deep learning ...,We instead use the term parameters. In deep le...


In [171]:
# create a new df with the questions and answers
gpt_df = pd.DataFrame()

# add new columns to the df
gpt_df['prompt'] = ''
gpt_df['completion'] = ''
gpt_df['question_id'] = 0

question_i = 0

# set the initial entry to empty values and question_id to 0
gpt_df.loc[question_i, 'prompt'] = ''
gpt_df.loc[question_i, 'completion'] = ''
gpt_df.loc[question_i, 'question_id'] = 0

with open('gpt-chat-log-cleaned.txt', 'r') as f:
    lines = f.readlines()

    for i, line in enumerate(lines):
        # if it starts with "Q: ", then it is a question
        if line.startswith("Q: "):
            # add a new entry to the df with the question 
            gpt_df = gpt_df.append({'prompt': line[3:].strip(), 'completion': '', 'question_id': question_i}, ignore_index=True)
            
        # otherwise, append it to the answer for the current question
            question_i += 1
            # print(question_i)
        else:
            # add the answer to the df at the index question_i 
            # print(question_i)
            # print(gpt_df.iloc[question_i]['answer'])
            gpt_df.at[question_i, 'completion'] += line.strip() + ' '


gpt_df.head()
print(len(gpt_df))

# save to csv 
# gpt_df.to_csv('gpt-qa.csv', index=False)

# drop the question_id column
gpt_df = gpt_df.drop(columns=['question_id'])
# remove rows with empty questions
gpt_df = gpt_df[gpt_df['prompt'] != '']
gpt_df = gpt_df[gpt_df['completion'] != '']
gpt_df = gpt_df.dropna()

# replace all triple spaces with '\n\n' 
gpt_df['completion'] = gpt_df['completion'].apply(lambda x: x.replace('  ', ' \n\n'))

gpt_df['completion'] = gpt_df['completion'].apply(lambda x: x.replace('```', '\n```\n'))


# save to csv without the indices 
gpt_df.to_csv('gpt-qa-df.csv', index=False)


57


# OpenAI Fine-tuning API 

ok now let's combine these two and submit to the openai finetuning api 
-  first i'll upload the GPT chat one 
-  then i'll upload the fastai one

## Now let's format it for finetuning API 

In [ ]:
# load the csv file for more editing
gpt_loaded_df = pd.read_csv('gpt-qa-df.csv')
print(gpt_loaded_df.head())

# - Based on your file extension, your file is formatted as a CSV file
# - Your file contains 56 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
# - Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
# - All completions end with suffix ` `
#   WARNING: Some of your completions contain the suffix ` ` more than once. We suggest that you review your completions and add a unique ending
# - The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

# add a space to the end of each prompt
# Each prompt should end with a fixed separator to inform the model when the prompt ends and the completion begins. A simple separator which generally works well is \n\n###\n\n. The separator should not appear elsewhere in any prompt.

gpt_loaded_df['prompt'] = gpt_loaded_df['prompt'].apply(lambda x: x + '\n\n###\n\n')

# start each completion with a space 
# Each completion should start with a whitespace due to our tokenization, which tokenizes most words with a preceding whitespace.
# Each completion should end with a fixed stop sequence to inform the model when the completion ends. A stop sequence could be \n, ###, or any other token that does not appear in any completion.

gpt_loaded_df['completion'] = gpt_loaded_df['completion'].apply(lambda x: ' ' + x + '\n<+++>\n')


# save csv 

# gpt_loaded_df.to_csv('gpt-qa-formatted.csv', index=False)

# split the df into two dfs, one with the first 80% of data and one with the last 20% of data
gpt_train_df, gpt_valid_df = np.split(gpt_loaded_df.sample(frac=1), [int(.8*len(gpt_loaded_df))])

# save the train and test dfs to csv files
gpt_train_df.to_csv('gpt-qa-train.csv', index=False)
gpt_valid_df.to_csv('gpt-qa-valid.csv', index=False)


                                              prompt  \
0  whats the difference between ; and && in terminal   
1                            how to auto jump in cli   
2  how to list all files even hidden files in ter...   
3         how to list files in human readable format   
4               how to list details of files with ls   

                                          completion  
0  In the terminal, `;` and `&&` are used to sepa...  
1  To enable auto jumping in the CLI, you can use...  
2  To list all files, including hidden files, in ...  
3  To list files in human readable format in the ...  
4  To list details about files in the terminal us...  


# How to format the API call to the fine tuned model 

### format the prompt from the user before sending it 
After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string 

`\n\n###\n\n` 

for the model to start generating completions, rather than continuing with the prompt. 

### param in API call for stop sequence 
Make sure to include 

`stop=[" \n<+++>\n"]` 

so that the generated texts ends at the expected place.

### Format the output CSV file in terminal into a JSONL: 

openai tools fine_tunes.prepare_data -f <LOCAL_FILE>


In [ ]:
filename="gpt-qa-valid.csv"

In [ ]:
# !y| openai tools fine_tunes.prepare_data -f {filename}

In [ ]:
# openai tools fine_tunes.prepare_data -f {filename}

# Create the fine-tuned model 
- in CLI
- using the files we created earlier

In [ ]:
### in terminal: 

openai api fine_tunes.create \
-t gpt-qa-train_prepared.jsonl \
-v gpt-qa-valid_prepared.jsonl \
-m "davinci" \
--suffix "gpt-ml-qa-pairs-A"

SyntaxError: invalid syntax (819037427.py, line 3)

In [129]:
### in terminal: 

# smaller learning rate because smaller batch 

openai api fine_tunes.create \
-t gpt-qa-train-formatted_prepared.jsonl \
-v gpt-qa-valid-formatted_prepared.jsonl \
-m "davinci" \
-learning_rate_multiplier 0.05 \
--suffix "gpt-ml-qa-pairs-B" \
    

SyntaxError: invalid syntax (2530784851.py, line 5)

In [130]:
# i could generate over several hyperparameters, 
# create a function that makes the call 
# saves the output 
# gets the finetuned model id
# and then calls the next function with the finetuned model id as an argument


# can i create a df of openai models and then iterate through it?
# or do i need to create a function that takes the model id as an argument?
# or do i need to create a function that takes the model id as an argument and then calls the next function with the finetuned model id as an argument?

# i think i need to create a function that takes the model id as an argument and then calls the next function with the finetuned model id as an argument

# function: takes in a dataset (just training set for now for simplicity) 
# outputs a finetuned model "return" 



# === Abstracting the functions to create a fine-tuned model === 

In [225]:
# Create finetuned model 


import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

def upload_file_openai(filename): 
    r = openai.File.create(
        file=open(filename, "rb"),
        purpose='fine-tune'
    )
    return r


def create_finetuned_model(train_file_id, valid_file_id='', model="davinci", learning_rate_multiplier='', n_epochs='', suffix=''):

    # create a finetuned model
    finetuned_model_response = openai.FineTune.create(
        training_file=train_file_id,
        # validation_file=valid_file_id,
        model=model,
        learning_rate_multiplier=learning_rate_multiplier,
        n_epochs=n_epochs,
        suffix=suffix,
    )

    return finetuned_model_response



In [ ]:
_train_file_id = upload_file_openai('gpt-qa-train-formatted_prepared.jsonl')['id']
_valid_file_id = upload_file_openai('gpt-qa-valid-formatted_prepared.jsonl')['id']

ftm = create_finetuned_model(
    train_file_id=_train_file_id,
    valid_file_id=_valid_file_id,
    model="davinci",
    learning_rate_multiplier=0.05,
    n_epochs=1,
    suffix="gpt-ml-qa-pairs-C",
)

ftm_id = ftm['id']

In [226]:
###################################
# Create the file_to_finetuned_model function 

# Simplify all the above 
""""
file_to_finetuned_model

Inputs: 
    filename: string, name of the file to be uploaded to openai
    model: string, name of the model to be finetuned
    learning_rate_multiplier: float, learning rate multiplier
    n_epochs: int, number of epochs
    suffix: string, suffix to be added to the finetuned model name

Outputs: 
    finetuned_model_id: string, id of the finetuned model

"""

# takes in .jsonl file, returns a finetuned model response 

def file_to_finetuned_model(train_file, valid_file='', model="davinci", learning_rate_multiplier=0.1, n_epochs=4, suffix=''):

    # upload file to openai
    if train_file:
        assert(train_file.endswith('.jsonl')), "train_file must be a .jsonl file"
        _train_file_id = upload_file_openai(train_file)['id']
    else:
        FileNotFoundError('Must include a train_file')

    if valid_file != '':
        assert(valid_file.endswith('.jsonl')), "valid_file must be a .jsonl file"
        _valid_file_id = upload_file_openai(valid_file)['id']
    else: 
        _valid_file_id = ''

    # create a finetuned model
    finetuned_model_response = create_finetuned_model(
        train_file_id=_train_file_id,
        # valid_file_id=_valid_file_id,
        model=model,
        learning_rate_multiplier=learning_rate_multiplier,
        n_epochs=n_epochs,
        suffix=suffix,
    )

    return finetuned_model_response



In [238]:
# TEST
# test the file_to_finetuned_model function


banana_file = "gpt-qa-train-formatted_prepared.jsonl"
banana_suffix="qa-train-iterate"

ftm = file_to_finetuned_model(
    train_file=banana_file,
    suffix=banana_suffix + "-z"
)

ftm_id = ftm['id']
ftm_name=ftm['fine_tuned_model']
print(ftm_name)
print (ftm)

# ftm is only available after the model has been created!!!!

None
{
  "created_at": 1668949486,
  "events": [
    {
      "created_at": 1668949486,
      "level": "info",
      "message": "Created fine-tune: ft-qodKCfHvQpCRAdFPYzXowY22",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-qodKCfHvQpCRAdFPYzXowY22",
  "model": "davinci",
  "object": "fine-tune",
  "organization_id": "org-5RSpJP9M5vC6iGBmX0vRfiMp",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 50561,
      "created_at": 1668949486,
      "filename": "file",
      "id": "file-J9K60guYJAQ3KkGglz8h4wdy",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "status_details": null
    }
  ],
  "updated_at": 1668949486,
  "validation_files": []
}


In [166]:
ftm_id

'ft-CgHrXIX0OF8EsJhWLiOGUQvp'

In [250]:
# HYPERPARAMETER EXPLORATION 

# create a set of various hyperparameters to iterate through
# create a ftmodel for each one 
# save the ftmodel id to a list
# then iterate through the list and generate responses to the test-prompts for each one
# save the responses to a list
# then compare the responses to the test answers
import time 

# wrap this in tqdm to show progress
from tqdm import tqdm 


currsuffix = banana_suffix+suffix
currsuffix = "curie:ft-personal-2022-11-20-08-38-46"

def create_ftm_suffix(base,**kwargs):
    str = base + ":" + "_".join([f"{k}-{v}, " for k,v in kwargs.items()])
    return str

def already_exists_ftm(suffix):
    all_ft_models = openai.FineTune.list()['data']

    # this checks if the current name is already in the list of ft models
    # any( ) returns True if any element in the list is True
    exists = any([True for i in all_ft_models if suffix in i['fine_tuned_model']])
    print('ftmodel already exists: ', suffix)
    return exists
 

banana_file = "gpt-qa-train-formatted_prepared.jsonl"

base_suffix="qa-train-iterate"
# create a set of various hyperparameters to iterate through
# epochs=[1,2,3,4]
# learning_rate_multiplier=[0.1, 0.05, 0.01, 0.005, 0.001]
epochs=[1,2]
learning_rate_multiplier=[0.1, 0.05, 0.01]

suffixes=['A', 'B', 'C', 'D', 'E']

ftmodels=pd.DataFrame(columns=['id', 'epochs', 'learning_rate_multiplier', 'suffix'])

ftmodelcompletions = pd.DataFrame(columns=['id', 'ftm_name', 'epochs', 'learning_rate_multiplier', 'suffix', 'prompt','completion'])

# create a ftmodel for each one

for epoch in tqdm(epochs):
# for epoch in epochs:

    for lrm in tqdm(learning_rate_multiplier):

        #check if the current model already exists. if so, skip it
        curr_suffix = create_ftm_suffix(base_suffix,lrm=lrm, epoch=epoch)

        # if already_exists_ftm(curr_suffix): continue

        # @TODO actually this should say: 
        # if already exists, then don't create the finetuned model, just get the id
        # if doesn't exist then create it, and then get the id 

        # create a ft model with these parameters 
        ftm = file_to_finetuned_model(
            train_file=banana_file,
            suffix=curr_suffix,
            n_epochs=epoch,
            learning_rate_multiplier=lrm
        )

        ftm_id = ftm['id']

        print(f"\n\n======================= Initiated ftm:  ======================= \n ftm_id: {ftm_id}. \n Now we wait for the model to train... \n")

        # wait for the ft model to finish training
        current_ft_model_is_ready = await_ft_ready(ftm_id)
        if not current_ft_model_is_ready: 
            print(f"finetuned model timed out: {ftm_id}")
            continue

        # the model has passed the await test, so it can be called now
        # it's also been given a name, so it can be called by ftm_name now
        # ftm_name = ftm['fine_tuned_model'] # the name is stored as ftm['fine_tuned_model']

        ftm = openai.FineTune.retrieve(ftm_id) # need to retrieve the fresh version with its name! 
        
        ftm_name = ftm['fine_tuned_model']

        print (ftm_name, curr_suffix)
        #also we could derive the name from the suffix! although they add a timestamp to the end of the name, which we'd want to emulate 


        print (f"{ftm_id} finetuned model is now ready for testing. \nIts name is: {ftm_name}\n")

        # append this model to the df 
        ftmodels = ftmodels.append({'id': ftm_id, 'ftm_name': ftm_name, 'epochs': epoch, 'learning_rate_multiplier': lrm, 'suffix': curr_suffix}, ignore_index=True)

        # now test every prompt with every ftmodel
        with open ('test-prompts.txt') as f:
            print(f"starting test-prompts on finetuned model {ftm_id}")
            prompts = f.readlines()
            for prompt in tqdm(prompts):
                # get the completion for this prompt
                completion = get_gpt_answer(prompt, ftm_name, printit=True)
                # append this completion to the df
                ftmodelcompletions = ftmodelcompletions.append(
                    {'id': ftm_id, 
                    'ftm_name': ftm_name,
                    'epochs': epoch, 
                    'learning_rate_multiplier': lrm, 
                    'suffix': curr_suffix, 
                    'prompt': prompt, 
                    'completion': completion}, 
                    ignore_index=True)
            print('\n\n\nCompleted test prompts for ftmodel: ', ftm_id)


ftmodels.to_csv('ftmodels.csv')
ftmodelcompletions.to_csv('ftmodelcompletions.csv')

print(ftmodels.head())
print(ftmodelcompletions.head())



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Initiated ftm: 
 ftm_id: ft-2jSFUg0jBJUslmfkCI5iDrST. 
 Now we wait for the model to train... 

waiting for ft-2jSFUg0jBJUslmfkCI5iDrST to train.
ft-2jSFUg0jBJUslmfkCI5iDrST has trained for 100 seconds so far.
ft-2jSFUg0jBJUslmfkCI5iDrST is ready!
{
  "created_at": 1668952253,
  "events": [
    {
      "created_at": 1668952253,
      "level": "info",
      "message": "Created fine-tune: ft-2jSFUg0jBJUslmfkCI5iDrST",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668952262,
      "level": "info",
      "message": "Fine-tune costs $0.35",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668952263,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668952266,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668952366,
      "level": "info",
      "message": "Completed epoch 1/

  0%|          | 0/6 [00:00<?, ?it/s]

creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-13-53-18 with prompt: What is the difference between a CNN and a DNN? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-13-53-18 with prompt: How many roads must a man walk down? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-13-53-18 with prompt: What does `DataLoaders` do in the fast.ai library? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-13-53-18 with prompt: Please write python code for a list comprehension


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-13-53-18 with prompt: Please write a bash script to save the output of a terminal command to a json file


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-13-53-18 with prompt: What are the steps for stochastic gradient descent? 

###





C

...

creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-01-epoch-1-2022-11-20-14-01-39 with prompt: What is the difference between a CNN and a DNN? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-01-epoch-1-2022-11-20-14-01-39 with prompt: How many roads must a man walk down? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-01-epoch-1-2022-11-20-14-01-39 with prompt: What does `DataLoaders` do in the fast.ai library? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-01-epoch-1-2022-11-20-14-01-39 with prompt: Please write python code for a list comprehension


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-01-epoch-1-2022-11-20-14-01-39 with prompt: Please write a bash script to save the output of a terminal command to a json file


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-01-epoch-1-2022-11-20-14-01-39 with prompt: What are the steps for stochastic gradient descent? 

###


  0%|          | 0/3 [00:00<?, ?it/s]

Initiated ftm: 
 ftm_id: ft-RIqxCLVFMKnDOhW7rSS38zKX. 
 Now we wait for the model to train... 

waiting for ft-RIqxCLVFMKnDOhW7rSS38zKX to train.
ft-RIqxCLVFMKnDOhW7rSS38zKX has trained for 100 seconds so far.
ft-RIqxCLVFMKnDOhW7rSS38zKX is ready!
{
  "created_at": 1668952954,
  "events": [
    {
      "created_at": 1668952954,
      "level": "info",
      "message": "Created fine-tune: ft-RIqxCLVFMKnDOhW7rSS38zKX",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668952962,
      "level": "info",
      "message": "Fine-tune costs $0.70",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668952962,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668952963,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668953065,
      "level": "info",
      "message": "Completed epoch 1/

  0%|          | 0/6 [00:00<?, ?it/s]

creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-2-2022-11-20-14-05-16 with prompt: What is the difference between a CNN and a DNN? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-2-2022-11-20-14-05-16 with prompt: How many roads must a man walk down? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-2-2022-11-20-14-05-16 with prompt: What does `DataLoaders` do in the fast.ai library? 


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-2-2022-11-20-14-05-16 with prompt: Please write python code for a list comprehension


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-2-2022-11-20-14-05-16 with prompt: Please write a bash script to save the output of a terminal command to a json file


###


creating call from davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-2-2022-11-20-14-05-16 with prompt: What are the steps for stochastic gradient descent? 

###





C

In [234]:
# HELPERS 

def get_ftmodel_name_from_id(id):
    ftmodel = openai.FineTune.retrieve(id)
    return ftmodel['fine_tuned_model']

def get_ftmodel_id_from_name(name):
    all_ft_models = openai.FineTune.list()['data']
    for ftmodel in all_ft_models:
        if name in ftmodel['fine_tuned_model']:
            return ftmodel['id']
    return None

print(get_ftmodel_name_from_id("ft-xXiSGiL0RrMUpKikJw09T1up"))
print(get_ftmodel_id_from_name("davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-12-50-36"))

davinci:ft-sandbox:qa-train-iterate-lrm-0-1-epoch-1-2022-11-20-12-50-36
ft-xXiSGiL0RrMUpKikJw09T1up


In [241]:
# openai.Model.list()

In [248]:
# GET FTM AWAIT READY 


# the finetuned model needs to finish training before we can use it
# so let's track its status and only make calls when its' ready 
# we can use the finetuned model id to check its status

def get_finetuned_model_status(finetuned_model_id):
    response = openai.FineTune.retrieve(id=finetuned_model_id)
    return response['status']

# check the status of the finetuned model
print(get_finetuned_model_status(ftm_id))

# now let's create a function that will wait until the finetuned model is ready
# and then return the response

def await_ft_ready(finetuned_model_id):

    patience = 40 # number of times to check the status of the finetuned model
    counter=0

    print(f"waiting for {finetuned_model_id} to train.")

    # print (openai.FineTune.retrieve(id=finetuned_model_id))

    while True:
        status = get_finetuned_model_status(finetuned_model_id)
        if status == 'succeeded':
            print(f"{finetuned_model_id} is ready!")
            print (openai.FineTune.retrieve(id=finetuned_model_id))

            return True
        else:
            # print(f"{finetuned_model_id} finetuned model is still training. {patience*10} seconds so far.")
            counter +=1
            if counter % 10 == 0: 
                print(f"{finetuned_model_id} has trained for {counter*10} seconds so far.")
            if counter > patience: 
                # print that the current timed out
                print(f"{finetuned_model_id} finetuned model has taken longer than {patience*10} to train. Calling exceeds patience variable.")
                print (openai.FineTune.retrieve(id=finetuned_model_id))
                return False
            time.sleep(10) 

# now let's test it out
print(await_ft_ready(ftm_id))

succeeded
waiting for ft-6RlUCVJ6XrFOiKJ7yEzP8vbP to train.
ft-6RlUCVJ6XrFOiKJ7yEzP8vbP is ready!
{
  "created_at": 1668951167,
  "events": [
    {
      "created_at": 1668951167,
      "level": "info",
      "message": "Created fine-tune: ft-6RlUCVJ6XrFOiKJ7yEzP8vbP",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668951170,
      "level": "info",
      "message": "Fine-tune costs $0.35",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668951170,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668951173,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668951274,
      "level": "info",
      "message": "Completed epoch 1/1",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1668951306,
      "level": "info",
      "message": "Uploaded model: davinci:ft


We attach a result file to each job once it has been completed. This results file ID will be listed when you retrieve a fine-tune, and also when you look at the events on a fine-tune. You can download these files.

The _results.csv file contains a row for each training step, where a step refers to one forward and backward pass on a batch of data. 

openai api fine_tunes.results -i <YOUR_FINE_TUNE_JOB_ID>


# Terminal outputs 




(ml) SF-mbp:gpt-fine-tuning stephen$ openai api fine_tunes.create \
> -t gpt-qa-train_prepared.jsonl \
> -v gpt-qa-valid_prepared.jsonl
-m "davinci" \
--suffix "gpt-ml-qa-pairs-A"
Upload progress: 100%|███████████████████████| 50.8k/50.8k [00:00<00:00, 22.6Mit/s]
Uploaded file from gpt-qa-train_prepared.jsonl: file-5fYU80lXy5FQgfmfVU56k8nb
Upload progress: 100%|███████████████████████| 12.2k/12.2k [00:00<00:00, 8.80Mit/s]
Uploaded file from gpt-qa-valid_prepared.jsonl: file-99tOIqCaPbNwL90bI1K1yq23
Created fine-tune: ft-zHk8gGvZdaBNRVheXq5fZFqt
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-11-20 00:36:48] Created fine-tune: ft-zHk8gGvZdaBNRVheXq5fZFqt
[2022-11-20 00:36:52] Fine-tune costs $0.14
[2022-11-20 00:36:52] Fine-tune enqueued. Queue number: 0
[2022-11-20 00:36:54] Fine-tune started
[2022-11-20 00:37:54] Completed epoch 1/4
[2022-11-20 00:38:06] Completed epoch 2/4
[2022-11-20 00:38:18] Completed epoch 3/4
[2022-11-20 00:38:30] Completed epoch 4/4
[2022-11-20 00:38:46] Uploaded model: curie:ft-personal-2022-11-20-08-38-46
[2022-11-20 00:38:47] Uploaded result file: file-EZy5duulRUswzYdqj6LuW76a
[2022-11-20 00:38:47] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2022-11-20-08-38-46 -p <YOUR_PROMPT>
(ml) SF-mbp:gpt-fine-tuning stephen$ -m "davinci" \
> --suffix "gpt-ml-qa-pairs-A"
-bash: -m: command not found

### Trying again 

$ openai api fine_tunes.create \
> -t gpt-qa-train_prepared.jsonl \
> -v gpt-qa-valid_prepared.jsonl \
> -m "davinci" \
> --suffix "gpt-ml-qa-pairs-A"
Found potentially duplicated files with name 'gpt-qa-train_prepared.jsonl', purpose 'fine-tune' and size 50782 bytes
file-5fYU80lXy5FQgfmfVU56k8nb
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway:
Upload progress: 100%|███████████████████████████████| 50.8k/50.8k [00:00<00:00, 21.8Mit/s]
Uploaded file from gpt-qa-train_prepared.jsonl: file-XCo6Sza9cVyfIWjPmoatpK3s
Found potentially duplicated files with name 'gpt-qa-valid_prepared.jsonl', purpose 'fine-tune' and size 12227 bytes
file-99tOIqCaPbNwL90bI1K1yq23
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway:
Upload progress: 100%|███████████████████████████████| 12.2k/12.2k [00:00<00:00, 6.49Mit/s]
Uploaded file from gpt-qa-valid_prepared.jsonl: file-HbOaJkGRRu4hp4C8DQ4PtZ0t
Created fine-tune: ft-xNzKl8ORDdASWIALy5WoSLcY
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-11-20 00:45:39] Created fine-tune: ft-xNzKl8ORDdASWIALy5WoSLcY
[2022-11-20 00:45:46] Fine-tune costs $1.42
[2022-11-20 00:45:47] Fine-tune enqueued. Queue number: 0
[2022-11-20 00:45:48] Fine-tune started
[2022-11-20 00:47:30] Completed epoch 1/4
[2022-11-20 00:47:50] Completed epoch 2/4
[2022-11-20 00:48:10] Completed epoch 3/4
[2022-11-20 00:48:30] Completed epoch 4/4
[2022-11-20 00:49:22] Uploaded model: davinci:ft-sandbox:gpt-ml-qa-pairs-a-2022-11-20-08-49-22
[2022-11-20 00:49:23] Uploaded result file: file-rGIPU3ndPAZXVRSBT4xhjDKy
[2022-11-20 00:49:23] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-sandbox:gpt-ml-qa-pairs-a-2022-11-20-08-49-22 -p <YOUR_PROMPT>


# Now, test the fine-tuned model with prompts 

In [123]:
import openai 
from dotenv import load_dotenv
load_dotenv()
# set the api key
openai.api_key = os.getenv("OPENAI_API_KEY")


In [245]:
# davinci:ft-sandbox:gpt-ml-qa-pairs-b-2022-11-20-09-29-25

# call the openai api with a prompt and get a response

ftm_name = "davinci:ft-sandbox:gpt-ml-qa-pairs-b-2022-11-20-09-29-25"

prompt = "What is the difference between a neural network and a deep learning model?" 

def get_gpt_answer(prompt, ftm_name_, printit=False): 

    prompt += "\n\n###\n\n"

    if printit:
        print(f"creating call from {ftm_name_} with prompt: {prompt}")

    response = openai.Completion.create(
        model=ftm_name_,
        # engine="text-davinci-002", 
        prompt=prompt,
        temperature=0.9,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0.6,
        stop=[" \n<+++>\n"]
    )

    return response['choices'][0]['text']


print(get_gpt_answer(prompt, ftm_name))


 In general, "deep learning" refers to the use of many layers of computational units (i.e. neurons) in a neural network or other machine learning model. 

 In other words, a deep learning model is a neural network or other machine learning model that uses many layers of computational units. 

 In practice, the terms "deep learning" and "neural network" are often used interchangeably, since deep learning is typically used for building neural networks. However, the terms can be used more specifically to refer to different aspects of neural network or deep learning models. For example, a "deep neural network" may be used to refer to a neural network with many layers of hidden units or neurons, while the term "deep learning" may be used to refer to the use of GPUs or other hardware for performing large amounts of parallel processing on the data. ‍ ‍


In [246]:
# Test prompts 

# for each line in the file, call the openai api and get a response
with open('test-prompts.txt', 'r') as f: 
    lines = f.readlines()
    for i, line in enumerate(lines): 
        answer = get_gpt_answer(line, ftm_name)
        print(f'Question {i}: {line}')
        print(f'Answer {i}: {answer} \n\n\n')

Question 0: What is the difference between a CNN and a DNN? 

Answer 0:  
 There are several differences between Convolutional Neural Networks (CNNs) and Deep Neural Networks (DNNs). 

 Firstly, while both CNNs and DNNs use neural networks, the structure of the neural networks in a CNN is different from that of a DNN. In a CNN, the layers of the neural network are arranged in a "convolutional" manner, with the output of each layer being connected to the input of the next layer via a small number of hidden units. In contrast, the layers of a DNN are arranged in a "feedforward" manner, with the output of each layer being connected to the input of the next layer via a large number of hidden units. 

 Secondly, while CNNs typically use a fixed number of filters per layer, DNNs use a variable number of filters per layer, determined by the optimization algorithm used to train the network. This is known as "feature scaling" in DNNs. Thirdly, while CNNs typically use a fixed number of hidden u

KeyboardInterrupt: 

In [ ]:
def bar(**kwargs):
    for key, value in kwargs.items():
        print("{} = {}".format(key, value))
    myb = kwargs["b"] if "b" in kwargs
    print(myb)
        

bar(a=1, b=2, c=3)
